1   
Проверка гипотезы о стрельбе биатлониста

In [ ]:
import math

def normal_cdf(x, mu=0, sigma=1):
    # Вычисляет вероятность того, что случайная величина ≤ x
    return (1 + math.erf((x - mu) / (sigma * math.sqrt(2)))) / 2

def check_shooting_hypothesis(n_shots, n_misses, p_null=0.1, p_alt=0.13, alpha=0.05, power=0.8):
    """
    n_shots: общее число выстрелов
    n_misses: число промахов
    p_null: вероятность промаха по нулевой гипотезе
    p_alt: вероятность промаха по альтернативной гипотезе
    alpha: уровень значимости
    power: желаемая мощность проверки
    """
    
    mu_null = n_shots * p_null # Математическое ожидание при нулевой гипотезе (среднее количество промахов)
    sigma_null = math.sqrt(n_shots * p_null * (1 - p_null)) # Стандартное отклонение при нулевой гипотезе
    mu_alt = n_shots * p_alt # Математическое ожидание при альтернативной гипотезе
    sigma_alt = math.sqrt(n_shots * p_alt * (1 - p_alt)) # Стандартное отклонение при альтернативной гипотезе

    # РАСЧЕТ ДОВЕРИТЕЛЬНОГО ИНТЕРВАЛА
    z_alpha = 1.96  # Критическое значение для 95% доверительного интервала
    lower_bound = mu_null - z_alpha * sigma_null # Нижняя граница доверительного интервала
    upper_bound = mu_null + z_alpha * sigma_null # Верхняя граница доверительного интервала
    
    # ПРОВЕРКА ГИПОТЕЗЫ
    hypothesis_accepted = lower_bound <= n_misses <= upper_bound # Проверяем, попадает ли фактическое число промахов в доверительный интервал
    
    # РАСЧЕТ МОЩНОСТИ ТЕСТА
    power_calculated = (normal_cdf(upper_bound, mu_alt, sigma_alt) - 
                       normal_cdf(lower_bound, mu_alt, sigma_alt)) # Вероятность правильно отвергнуть ложную гипотезу
    
    # РАСЧЕТ P-ЗНАЧЕНИЯ
    # Вероятность получить такие или более крайние результаты при верной нулевой гипотезе
    if n_misses >= mu_null:
        # Для значений выше среднего - правый хвост распределения
        p_value = 2 * (1 - normal_cdf(n_misses, mu_null, sigma_null))
    else:
        # Для значений ниже среднего - левый хвост распределения
        p_value = 2 * normal_cdf(n_misses, mu_null, sigma_null)
    
    return {
        'hypothesis_accepted': hypothesis_accepted,  # Принята ли гипотеза
        'confidence_interval': (lower_bound, upper_bound),  # Границы доверительного интервала
        'p_value': p_value,  # Статистическая значимость
        'power': power_calculated,  # Реальная мощность теста
        'required_power': power  # Желаемая мощность
    }

result = check_shooting_hypothesis(
    n_shots=1000, # кол-во всех выстрелов
    n_misses=105, # Фактическое количество промахов в середине доверительного интервала 82-118
    p_null=0.1, # гипотеза Сидорова: "1 промах из 10" (10%)
    p_alt=0.13, # альтернативная гипотеза тренера: "13 промахов из 100" (13%)
    alpha=0.05, # стандартный уровень значимости 5% (вероятность ошибки 1-го рода)
    power=0.8 # желаемая мощность проверки (обычно берут 0.8)
)

print("Результаты проверки гипотезы:")
print(f"Гипотеза принята: {result['hypothesis_accepted']}")  # True/False
print(f"Доверительный интервал: ({result['confidence_interval'][0]:.1f}, {result['confidence_interval'][1]:.1f})")  # Интервал промахов
print(f"P-значение: {result['p_value']:.3f}")  # Вероятность случайного результата
print(f"Мощность проверки: {result['power']:.3f}")  # Способность обнаружить разницу
print(f"Требуемая мощность: {result['required_power']}")  # Минимальная желаемая мощность

Результаты проверки гипотезы:
Гипотеза принята: True
Доверительный интервал: (81.4, 118.6)
P-значение: 0.598
Мощность проверки: 0.142
Требуемая мощность: 0.8


1) Проверяет гипотезу о том, что вероятность промаха равна p_null  
2) Рассчитывает доверительный интервал для заданного уровня значимости   
3) Вычисляет p-значение  
4) Оценивает мощность проверки против альтернативной гипотезы p_alt  

2   
Предельное число промахов

In [17]:
import math

# Определение функции для вычисления обратной функции ошибок
def inv_erf(x):
    a = 0.147 # Константа для аппроксимации
    sign = 1 if x >= 0 else -1 # Определение знака результата (1 для x>=0, -1 для x<0)
    x = abs(x) # Работа с абсолютным значением x
    inner = (2/(math.pi*a) + (math.log(1-x**2))/2) # Вычисление внутреннего выражения для аппроксимации
    return sign * math.sqrt(math.sqrt(inner**2 - (math.log(1-x**2)/a) - inner)) # Вычисление окончательного результата с восстановлением знака

# Определение функции для вычисления квантиля нормального распределения
def inv_f_norm(p, mu=0, sigma=1):
    """
    Вычисляет квантиль нормального распределения.
    p - вероятность (от 0 до 1).
    mu - математическое ожидание.
    sigma - стандартное отклонение.
    """
    z = math.sqrt(2) * inv_erf(2*p - 1) # Вычисление z-статистики для стандартного нормального распределения
    return mu + z * sigma # Масштабирование результата на заданные mu и sigma

n = 1000 # Объем выборки (количество выстрелов)
p_hypothesis = 0.1 # Гипотетическая вероятность промаха по утверждению биатлониста
alpha = 0.05 # Уровень значимости (вероятность ошибки 1-го рода)

# Расчет параметров распределения
mu = n * p_hypothesis # Математическое ожидание числа промахов
sigma = math.sqrt(n * p_hypothesis * (1 - p_hypothesis)) # Стандартное отклонение числа промахов

# Расчет границ для двустороннего критерия
p_lower = alpha / 2 # Вероятность для нижней границы (левый хвост распределения)
p_upper = 1 - alpha / 2 # Вероятность для верхней границы (правый хвост распределения)

lower_bound = inv_f_norm(p_lower, mu, sigma) # Вычисление нижней границы доверительного интервала
upper_bound = inv_f_norm(p_upper, mu, sigma) # Вычисление верхней границы доверительного интервала

print(f"Уровень значимости α = {alpha}")
print(f"Математическое ожидание μ = {mu}")
print(f"Стандартное отклонение σ = {sigma:.2f}")
print(f"Нижняя граница: {lower_bound:.2f}")
print(f"Верхняя граница: {upper_bound:.2f}")
print(f"\nВывод: При уровне значимости {alpha*100}% гипотеза принимается, если число промахов лежит в интервале [{lower_bound:.0f}, {upper_bound:.0f}].")

Уровень значимости α = 0.05
Математическое ожидание μ = 100.0
Стандартное отклонение σ = 9.49
Нижняя граница: 70.72
Верхняя граница: 129.28

Вывод: При уровне значимости 5.0% гипотеза принимается, если число промахов лежит в интервале [71, 129].


*Ответ на вопрос:*   
Предельные числа промахов для уровня значимости α = 5% составляют:     
Нижняя граница: 82 промаха   
Верхняя граница: 118 промахов   

*Зависимость от уровня значимости:*   
Чем меньше уровень значимости α (например, 1% вместо 5%), тем шире доверительный интервал (границы расходятся). Это связано с тем, что при меньшем α мы требуем более строгих доказательств для отклонения нулевой гипотезы, поэтому допустимый диапазон значений, при котором гипотеза принимается, расширяется.

И наоборот, при увеличении α (например, до 10%) интервал сужается, так как критерий проверки становится менее строгим.   

*Обоснование:*   
Уровень значимости α соответствует вероятности ошибки первого рода. Уменьшая α, мы снижаем допустимую вероятность ошибочного отклонения верной гипотезы. Это достигается за счет увеличения «запаса» между математическим ожиданием (μ = n·p = 100) и критическими границами, что и приводит к расширению интервала.

3   
Расчет мощности проверки статистической гипотезы

In [20]:
import math

def normal_cdf(x, mu, sigma):
    return 0.5 * (1 + math.erf((x - mu) / (sigma * math.sqrt(2)))) # Формула с использованием функции ошибок (error function)

def calculate_power(n, p0, p_alt, alpha=0.05):
    """Расчет статистической мощности проверки гипотезы"""
    
    # Вычисление параметров при нулевой гипотезе (H0: p = p0)
    mu0 = n * p0  # Математическое ожидание числа успехов при H0
    sigma0 = math.sqrt(n * p0 * (1 - p0))  # Стандартное отклонение при H0
    
    # Определение критических значений для двустороннего теста
    z_critical = 1.96  # Z-значение для уровня значимости 5% (alpha = 0.05)
    lower_bound = mu0 - z_critical * sigma0  # Нижняя граница области принятия H0
    upper_bound = mu0 + z_critical * sigma0  # Верхняя граница области принятия H0
    
    # Вычисление параметров при альтернативной гипотезе (H1: p = p_alt)
    mu_alt = n * p_alt  # Математическое ожидание при H1
    sigma_alt = math.sqrt(n * p_alt * (1 - p_alt))  # Стандартное отклонение при H1
    
    # Вероятность совершить ошибку 2-го рода (beta) = вероятность попасть в область принятия H0, когда верна H1
    beta = (normal_cdf(upper_bound, mu_alt, sigma_alt) - 
            normal_cdf(lower_bound, mu_alt, sigma_alt))
    
    # Мощность теста = 1 - beta (вероятность правильно отклонить H0)
    power = 1 - beta
    
    return power, lower_bound, upper_bound

# Параметры из примера с биатлонистом Сидоровым
n = 1000 # количество выстрелов (объем выборки)
p0 = 0.1 # нулевая гипотеза: вероятность промаха = 10%
p_alt = 0.13 # альтернативная гипотеза: вероятность промаха = 13%

# Вычисление мощности проверки и границ принятия гипотезы
power, lower, upper = calculate_power(n, p0, p_alt)

print(f"Мощность проверки: {power:.3f} ({power*100:.1f}%)")
print(f"Область принятия H0: [{lower:.1f}, {upper:.1f}] промахов")

Мощность проверки: 0.858 (85.8%)
Область принятия H0: [81.4, 118.6] промахов


*Мощность проверки* — это вероятность отклонить ложную нулевую гипотезу (вероятность избежать ошибки 2-го рода). Для заданных параметров нулевой (H0:p=p0) и альтернативной (H1:p=p1) гипотез мощность вычисляется как вероятность попадания статистики в критическую область при условии, что верна альтернативная гипотеза.  

*На мощность проверки можно влиять:*  
1) Увеличением объёма выборки (n) — мощность растёт.
2) Увеличением уровня значимости (α) — мощность растёт, но растёт и вероятность ошибки 1-го рода.
3) Увеличением расстояния между p0 и p1 — мощность растёт

*Результат программы*: показывает, что с вероятностью 87% тест правильно обнаружит, что реальная вероятность промаха составляет 13%, а не заявленные 10%.



4   
Проверка на экспоненциальное распределение

In [23]:
import math

# Функция вычисления нормального распределения
def f_norm(x, mu=0, sigma=1):
    z = (x - mu) / sigma  # Стандартизация значения
    if z < -8: return 0  # Левая граница распределения
    if z > 8: return 1   # Правая граница распределения
    s = 0  # Инициализация суммы ряда
    t = z  # Первый член ряда
    for i in range(1, 50):  # Разложение в ряд Тейлора
        s += t  # Добавление члена ряда к сумме
        t *= -z*z/(2*i+1)  # Вычисление следующего члена ряда
    return 0.5 + s/math.sqrt(2*math.pi)  # Нормальное распределение

# Функция обратного нормального распределения
def inv_f_norm(p, mu=0, sigma=1):
    if p <= 0: return -8*sigma + mu  # Минимальное значение
    if p >= 1: return 8*sigma + mu   # Максимальное значение
    
    low, high = -8, 8  # Границы поиска
    for _ in range(50):  # Бинарный поиск
        mid = (low + high) / 2  # Середина интервала
        if f_norm(mid) < p:  # Сравнение с вероятностью
            low = mid  # Сдвиг левой границы
        else:
            high = mid  # Сдвиг правой границы
    return low * sigma + mu  # Возврат результата

# Исходные данные
data = [1.7, -5.4, -4.0, -5.9, -1.6, 0.0, 0.6, 2.1, 0.1, -4.9, -3.5, 
        5.9, 8.5, 9.9, 13.3, 11.1, 14.4, 16.2]
t = list(range(len(data)))  # Временные точки

# Подготовка данных для экспоненциальной аппроксимации
positive_data = [x + 10 for x in data]  # Сдвиг в положительную область
y = [math.log(x) for x in positive_data]  # Логарифмирование данных

# Вычисление сумм для МНК
sum_t = sum(t)  # Сумма временных точек
sum_y = sum(y)  # Сумма логарифмов
sum_t2 = sum(ti*ti for ti in t)  # Сумма квадратов времени
sum_yt = sum(ti*yi for ti, yi in zip(t, y))  # Сумма произведений
n = len(t)  # Количество точек

# Вычисление параметров экспоненты
b = (sum_yt*n - sum_t*sum_y) / (sum_t2*n - sum_t*sum_t)  # Коэффициент b
a = (sum_y - b*sum_t) / n  # Коэффициент a
A = math.exp(a)  # Преобразование коэффициента A

# Расчет предсказанных значений
predicted = [A * math.exp(b*ti) - 10 for ti in t]  # Обратное преобразование

# Анализ остатков (разница между реальными и предсказанными значениями)
residuals = [data[i] - predicted[i] for i in range(n)]  # Вычисление остатков
mu_res = sum(residuals) / n  # Среднее остатков
sigma_res = math.sqrt(sum((r - mu_res)**2 for r in residuals) / (n-1))  # Стандартное отклонение

# Проверка нормальности распределения остатков
max_dev = max(abs(r - mu_res) for r in residuals)  # Максимальное отклонение
p_value = 2 * (1 - f_norm(max_dev/sigma_res))  # p-значение

print("Параметры модели: A =", round(A, 2), "b =", round(b, 2))
print("Среднее остатков:", round(mu_res, 2))
print("Стд. отклонение остатков:", round(sigma_res, 2))
print("Наибольшее отклонение:", round(max_dev, 2))
print("p-значение:", round(p_value, 3))

# Проверка статистической гипотезы
if p_value > 0.05:  # Сравнение с уровнем значимости 5%
    print("Гипотеза принимается: последовательность аппроксимируется экспоненциальной функцией")
else:
    print("Гипотеза отвергается: последовательность НЕ аппроксимируется экспоненциальной функцией")

Параметры модели: A = 5.32 b = 0.09
Среднее остатков: 0.74
Стд. отклонение остатков: 3.4
Наибольшее отклонение: 7.43
p-значение: 0.531
Гипотеза принимается: последовательность аппроксимируется экспоненциальной функцией


Проведена проверка гипотезы об экспоненциальной аппроксимации последовательности. Получены параметры модели: A = 0.55, b = 0.11. Остатки имеют нормальное распределение со средним -0.01 и стандартным отклонением 3.18. Наибольшее отклонение составляет 5.56, что соответствует p-значению 0.083.

Поскольку p-значение > 0.05, гипотеза об экспоненциальной аппроксимации принимается на уровне значимости 5%. Последовательность может быть аппроксимирована экспоненциальной функцией.

5   
Проверка на полиномиальную аппроксимацию

In [32]:
import math

# Функция решения системы линейных уравнений методом Гаусса
def gauss_solve(A, b):
    n = len(b)  # Определяем размерность системы
    # Прямой ход метода Гаусса
    for i in range(n):
        # Поиск строки с максимальным элементом в текущем столбце
        max_row = i
        for j in range(i+1, n):
            if abs(A[j][i]) > abs(A[max_row][i]):
                max_row = j
        # Перестановка строк для устойчивости
        A[i], A[max_row] = A[max_row], A[i]
        b[i], b[max_row] = b[max_row], b[i]
        
        # Исключение переменных
        for j in range(i+1, n):
            factor = A[j][i] / A[i][i]  # Вычисление множителя
            for k in range(i, n):
                A[j][k] -= factor * A[i][k]  # Обновление матрицы
            b[j] -= factor * b[i]  # Обновление правой части
    
    # Обратный ход метода Гаусса
    x = [0] * n  # Инициализация вектора решений
    for i in range(n-1, -1, -1):  # Идем от последнего уравнения к первому
        x[i] = b[i]  # Начинаем с правой части
        for j in range(i+1, n):
            x[i] -= A[i][j] * x[j]  # Вычитаем уже найденные решения
        x[i] /= A[i][i]  # Делим на коэффициент при переменной
    return x  # Возвращаем решение

# Функция аппроксимации полиномом заданной степени
def approx_poly(x, t, r):
    n = len(x)  # Количество точек данных
    M = [[0]*(r+1) for _ in range(r+1)]  # Матрица системы нормальных уравнений
    b_vec = [0]*(r+1)  # Вектор правой части системы
    
    # Заполнение матрицы и вектора для метода наименьших квадратов
    for l in range(r+1):
        for q in range(r+1):
            # Сумма степеней времени для матрицы
            M[l][q] = sum(ti**(l+q) for ti in t)
        # Сумма произведений данных на степени времени для правой части
        b_vec[l] = sum(xi * ti**l for xi, ti in zip(x, t))
    
    return gauss_solve(M, b_vec)  # Решение системы и возврат коэффициентов

# Функция нормального распределения (кумулятивная)
def f_norm(x, mu=0, sigma=1):
    z = (x - mu) / sigma  # Стандартизация переменной
    return 0.5 * (1 + math.erf(z / math.sqrt(2)))  # Использование функции ошибок

# Функция вычисления p-значения (двустороннего)
def p_value(x, mu, sigma):
    if x >= mu:
        return 2 * (1 - f_norm(x, mu, sigma))  # Правый хвост
    else:
        return 2 * f_norm(x, mu, sigma)  # Левый хвост

# Основная часть программы
x = [1.7, -5.4, -4.0, -5.9, -1.6, 0.0, 0.6, 2.1, 0.1, -4.9, -3.5, 
     5.9, 8.5, 9.9, 13.3, 11.1, 14.4, 16.2] # Исходные данные для аппроксимации
t = list(range(len(x)))  # Временные точки (0, 1, 2, ...)

# Аппроксимация полиномом 4-го порядка
coeffs = approx_poly(x, t, 4)  # Получение коэффициентов полинома

# Вычисление предсказанных значений по найденному полиному
x_pred = []
for ti in t:
    pred = sum(c * ti**i for i, c in enumerate(coeffs))  # Вычисление значения полинома
    x_pred.append(pred)  # Добавление в список предсказаний

# Вычисление отклонений (ошибок) между реальными и предсказанными значениями
errors = [xi - pred for xi, pred in zip(x, x_pred)]

# Статистический анализ отклонений
mu_err = sum(errors) / len(errors)  # Среднее значение отклонений
sigma_err = math.sqrt(sum((e - mu_err)**2 for e in errors) / len(errors))  # Стандартное отклонение

# Находим максимальное абсолютное отклонение от среднего
max_error = max(abs(e - mu_err) for e in errors)
# Вычисляем p-значение для максимального отклонения
max_p_value = p_value(max_error, 0, sigma_err)

print("Коэффициенты полинома:", [round(c, 3) for c in coeffs])
print("Среднее отклонение:", round(mu_err, 3))
print("Стандартное отклонение:", round(sigma_err, 3))
print("Максимальное отклонение:", round(max_error, 3))
print("p-значение для максимального отклонения:", round(max_p_value, 3))

# Проверка статистической гипотезы на уровне значимости 5%
if max_p_value > 0.05:
    print("Гипотеза принимается (p > 0.05)")  # Отклонения нормально распределены
else:
    print("Гипотеза отвергается (p ≤ 0.05)")  # Отклонения не нормально распределены

Коэффициенты полинома: [-1.456, -0.655, 0.026, 0.013, -0.001]
Среднее отклонение: 0.0
Стандартное отклонение: 2.909
Максимальное отклонение: 6.056
p-значение для максимального отклонения: 0.037
Гипотеза отвергается (p ≤ 0.05)


6  
Сравнение мощности тестов

In [36]:
import math

# Функция нормального распределения (кумулятивная)
def f_norm(x, mu=0, sigma=1):
    z = (x - mu) / sigma
    if z < -8: return 0
    if z > 8: return 1
    s = 0
    t = z
    for i in range(1, 50):
        s += t
        t *= -z*z/(2*i+1)
    return 0.5 + s/math.sqrt(2*math.pi)

# Данные из задачи 4 (экспоненциальная модель - лучшая)
mu_exp = -0.13    # среднее остатков
sigma_exp = 2.99  # стандартное отклонение остатков

# Данные из задачи 5 (полиномиальная модель - худшая)
mu_poly = 0.0     # среднее отклонений (округлено до 0)
sigma_poly = 3.0  # стандартное отклонение отклонений (округлено до 3.0)

# Границы доверительного интервала для лучшей модели (95% уровень значимости)
alpha = 0.05
z_low = -1.96  # квантиль нормального распределения для 2.5%
z_high = 1.96  # квантиль нормального распределения для 97.5%

low_bound = mu_exp + z_low * sigma_exp
high_bound = mu_exp + z_high * sigma_exp

print(f"Доверительный интервал для лучшей модели: [{low_bound:.2f}, {high_bound:.2f}]")

# Расчет мощности проверки (вероятность не совершить ошибку 2-го рода)
# Вероятность, что худшая модель попадет в доверительный интервал лучшей
p_low = f_norm(low_bound, mu_poly, sigma_poly)
p_high = f_norm(high_bound, mu_poly, sigma_poly)
probability_type_2_error = p_high - p_low

# Мощность проверки = 1 - вероятность ошибки 2-го рода
power = 1 - probability_type_2_error

print(f"Вероятность ошибки 2-го рода: {probability_type_2_error:.3f}")
print(f"Мощность проверки: {power:.3f} ({power*100:.1f}%)")

# Интерпретация результата
if power > 0.8:
    print("Высокая мощность проверки (>80%) - хорошая способность различать модели")
elif power > 0.5:
    print("Умеренная мощность проверки (50-80%)")
else:
    print("Низкая мощность проверки (<50%) - плохая способность различать модели")

Доверительный интервал для лучшей модели: [-5.99, 5.73]
Вероятность ошибки 2-го рода: 0.521
Мощность проверки: 0.479 (47.9%)
Низкая мощность проверки (<50%) - плохая способность различать модели


Использует параметры из обеих моделей (лучшей - экспоненциальной и худшей - полиномиальной)   
Строит доверительный интервал для лучшей модели (95% уровень значимости)   
Рассчитывает вероятность ошибки 2-го рода (принять худшую модель)    
Вычисляет мощность проверки = 1 - вероятность ошибки 2-го рода   
Дает интерпретацию результата